In [8]:
%matplotlib widget
import ipympl
import json

import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import c as clight

import xpart as xp
import xtrack as xt

fname_line = './generate_models/sps_acceleration.json'
with open(fname_line, 'r') as fid:
     model_data = json.load(fid)

line = xt.Line.from_dict(model_data['line'])

Done loading line from dict.           


## No acceleration

Build the tracker and some particles:

In [9]:
line.build_tracker()

particles = xp.Particles(p0c=26e9, delta=np.linspace(-4e-3, 4e-3, 40))
line.track(particles, num_turns=500, turn_by_turn_monitor=True)

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


Let's start by plotting the longitudinal phase space without acceleration:

In [10]:
rec = line.record_last_track

## Plot:
fig, ax = plt.subplots()
fig.suptitle('Longitudinal phase space')

for ii in range(rec.x.shape[0]):
    mask = rec.state[ii, :]>0
    ax.plot(rec.zeta[ii, mask], rec.delta[ii, mask])

ax.grid(linestyle='--')
ax.set_xlabel('z [m]')
ax.set_ylabel(r'$\Delta p / p_0$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$\\Delta p / p_0$')

## With acceleration

Add an element that will increase the energy at the end of each turn:

In [11]:
line.unfreeze()  # to modify the line we discard the current tracker

T_rev = 23e-6 # 23 us
Delta_p0c = 400e9 / 10 * T_rev # energy increase per turn (400 GeV in 10 s)

energy_increase = xt.ReferenceEnergyIncrease(Delta_p0c=Delta_p0c)
line.append_element(energy_increase, 'energy_increase')

Build the tracker and track some particles

In [12]:
line.build_tracker()

particles = xp.Particles(p0c=26e9, delta=np.linspace(-4e-3, 4e-3, 40))
line.track(particles, num_turns=500, turn_by_turn_monitor=True)

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


Plot our phase space

In [13]:
rec = line.record_last_track

fig, ax = plt.subplots()
fig.suptitle('Longitudinal phase space')

for ii in range(rec.x.shape[0]):
    mask = rec.state[ii, :]>0
    ax.plot(rec.zeta[ii, mask], rec.delta[ii, mask])

ax.grid(linestyle='--')
ax.set_xlabel('z [m]')
ax.set_ylabel(r'$\Delta p / p_0$')

# Assume only first cavity is active
frequency = line.get_elements_of_type(xt.Cavity)[0][0].frequency
voltage = line.get_elements_of_type(xt.Cavity)[0][0].voltage
#Assuming proton and beta=1
stable_z = np.arcsin(Delta_p0c/voltage)/frequency/2/np.pi*clight

p_co = line.find_closed_orbit(particle_ref=xp.Particles.from_dict(model_data['particle']))

assert np.isclose(p_co.zeta, stable_z, atol=0, rtol=1e-2)

ax.axvline(x=stable_z, color='black', linestyle='--')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …